#Authentications & Libraries

In [8]:
import pandas as pd
import numpy as np
import re
import nltk
import tweepy
from textblob import TextBlob
from nltk.stem.snowball import SnowballStemmer

In [11]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


# Scrapping

In [2]:
def twt(dict, movie):
  CONSUMER_KEY ="xxxxxxxxxxxxxxxxxx"
  CONSUMER_SECRET = "xxxxxxxxxxxxxxxxxx"
  ACCESS_KEY = "xxxxxxxxxxxxxxxxxx"
  ACCESS_SECRET = "xxxxxxxxxxxxxxxxxx"
  auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
  auth.set_access_token(ACCESS_KEY, ACCESS_SECRET)
  api = tweepy.API(auth)
  corpus_tweets = api.search("{}".format(movie), count=300)
  
  j = 0
  for tweet in corpus_tweets:
    tweets = tweet.text
    blob = TextBlob(tweets)

    if blob.detect_language()=='en' :
     
      dict[j] = '{}'.format(tweets)
      j += 1
  return(dict)

In [3]:

dict = {}
twt(dict, 'Messi')

{0: 'RT @TrollFootball_6: The Trio Messi, Neymar &amp; Mbappe is too unfair to the others . The Guys will have the best creator in the World , the b…',
 1: 'RT @ClintsAyebare: Pedri in December trying to fill the Messi role https://t.co/SqgWTzkq4H',
 2: 'RT @shawtaee: Boys make fun of girls for talking about BTS all the time and then go on to talk about Messi leaving Barcelona for hours',
 3: 'RT @KahunaKome: He heard that Messi news and threw himself down the stairs',
 4: 'RT @andreaxfc: bless the journalist who asked messi to smile one last time in front of the barça crest💙❤️ https://t.co/guLZu3wPAT',
 5: 'RT @goal: Lionel Messi will undergo a medical at PSG tomorrow 🇫🇷 https://t.co/lrDi4UkNlm',
 6: '@trfcste @SirPELE_TheKING @vajevic @GaryLineker @FCBarcelona That is amazing. It’s like a cult with him fat worse t… https://t.co/g5QM9TPq9f',
 7: 'RT @FaruqBashar: Every cent Messi earned at Barcelona, he deserved it.',
 8: 'RT @TopboyNas: @goal Scenes when Messi meets Mbappe at PSG in 

In [4]:
# create dataframe
data = dict
df=pd.DataFrame.from_dict(data,columns=["tweet"] ,orient='index')

#Cleaning

In [5]:
#Pattern removing
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
        
    return input_txt

In [6]:
# Removing RT@USER:

df['tidy_tweet'] = np.vectorize(remove_pattern)(df['tweet'], "RT @[\w]*:")

# Removing hyperlinks

df['tidy_tweet'] = np.vectorize(remove_pattern)(df['tidy_tweet'], r'https?:\/\/\S*') 

# removing hashtags  

df['tidy_tweet'] = np.vectorize(remove_pattern)(df['tidy_tweet'], "#[A-Za-z0-9_]+") 


#removing Non english characters(Punctiuations etc)
df['tidy_tweet'] = df['tidy_tweet'].str.replace("[^a-zA-Z#]", " ")

df.head()

,tweet,tidy_tweet
0,"RT @TrollFootball_6: The Trio Messi, Neymar &a...",The Trio Messi Neymar amp Mbappe is too un...
1,RT @ClintsAyebare: Pedri in December trying to...,Pedri in December trying to fill the Messi role
2,RT @shawtaee: Boys make fun of girls for talki...,Boys make fun of girls for talking about BTS ...
3,RT @KahunaKome: He heard that Messi news and t...,He heard that Messi news and threw himself do...
4,RT @andreaxfc: bless the journalist who asked ...,bless the journalist who asked messi to smile...


In [9]:
# stemming

# Use English stemmer.
stemmer = SnowballStemmer("english")

df['stemmed'] = df['tidy_tweet'].str.split()
df['stemmed'] = df['stemmed'].apply(lambda x: [stemmer.stem(y) for y in x])
df['stemmed'] = df['stemmed'].apply(lambda x: ' '.join(x))

In [10]:
df

,tweet,tidy_tweet,stemmed
0,"RT @TrollFootball_6: The Trio Messi, Neymar &a...",The Trio Messi Neymar amp Mbappe is too un...,the trio messi neymar amp mbapp is too unfair ...
1,RT @ClintsAyebare: Pedri in December trying to...,Pedri in December trying to fill the Messi role,pedri in decemb tri to fill the messi role
2,RT @shawtaee: Boys make fun of girls for talki...,Boys make fun of girls for talking about BTS ...,boy make fun of girl for talk about bts all th...
3,RT @KahunaKome: He heard that Messi news and t...,He heard that Messi news and threw himself do...,he heard that messi news and threw himself dow...
4,RT @andreaxfc: bless the journalist who asked ...,bless the journalist who asked messi to smile...,bless the journalist who ask messi to smile on...
5,RT @goal: Lionel Messi will undergo a medical ...,Lionel Messi will undergo a medical at PSG to...,lionel messi will undergo a medic at psg tomorrow
6,@trfcste @SirPELE_TheKING @vajevic @GaryLineke...,trfcste SirPELE TheKING vajevic GaryLineke...,trfcste sirpel theke vajev garylinek fcbarcelo...
7,RT @FaruqBashar: Every cent Messi earned at Ba...,Every cent Messi earned at Barcelona he dese...,everi cent messi earn at barcelona he deserv it
8,RT @TopboyNas: @goal Scenes when Messi meets M...,goal Scenes when Messi meets Mbappe at PSG i...,goal scene when messi meet mbapp at psg in the...
9,@VIC_11s @goal I am a neymar fan but I totally...,VIC s goal I am a neymar fan but I totally...,vic s goal i am a neymar fan but i total disag...


#Polarity 

In [12]:
def pol(text):
  testimonial = TextBlob(text)
  return testimonial.sentiment.polarity

In [13]:
b=df['stemmed'].values.tolist()
#map(remove(x), lang)
z=map(lambda i:pol(i),b)

polarity=list(z)

df['polarity']=polarity

In [14]:
df

,tweet,tidy_tweet,stemmed,polarity
0,"RT @TrollFootball_6: The Trio Messi, Neymar &a...",The Trio Messi Neymar amp Mbappe is too un...,the trio messi neymar amp mbapp is too unfair ...,0.125000
1,RT @ClintsAyebare: Pedri in December trying to...,Pedri in December trying to fill the Messi role,pedri in decemb tri to fill the messi role,0.000000
2,RT @shawtaee: Boys make fun of girls for talki...,Boys make fun of girls for talking about BTS ...,boy make fun of girl for talk about bts all th...,0.300000
3,RT @KahunaKome: He heard that Messi news and t...,He heard that Messi news and threw himself do...,he heard that messi news and threw himself dow...,-0.155556
4,RT @andreaxfc: bless the journalist who asked ...,bless the journalist who asked messi to smile...,bless the journalist who ask messi to smile on...,0.150000
5,RT @goal: Lionel Messi will undergo a medical ...,Lionel Messi will undergo a medical at PSG to...,lionel messi will undergo a medic at psg tomorrow,0.000000
6,@trfcste @SirPELE_TheKING @vajevic @GaryLineke...,trfcste SirPELE TheKING vajevic GaryLineke...,trfcste sirpel theke vajev garylinek fcbarcelo...,0.000000
7,RT @FaruqBashar: Every cent Messi earned at Ba...,Every cent Messi earned at Barcelona he dese...,everi cent messi earn at barcelona he deserv it,0.000000
8,RT @TopboyNas: @goal Scenes when Messi meets M...,goal Scenes when Messi meets Mbappe at PSG i...,goal scene when messi meet mbapp at psg in the...,0.000000
9,@VIC_11s @goal I am a neymar fan but I totally...,VIC s goal I am a neymar fan but I totally...,vic s goal i am a neymar fan but i total disag...,0.000000


#Saving CSV

In [16]:
df.to_csv('messi.csv')
!cp messi.csv "drive/My Drive/"